# Pipeline for reconstruction point source from raw data

Auth: M. Gamboa

Data used for JCh 

* 130GHz-2019-04-18
* 140GHz-2019-04-10
* 150GHz-2019-03-31 o 150GHz-2019-04-06
* 160GHz-2019-04-08
* 170GHz-2019-04-14

**Measured data** taken from CCA: `/sps/qubic/Data/Calib-TD`

* If you are working in your personal computer, you can copy the dataset by running: 

&nbsp;&nbsp;&nbsp;&nbsp;`scp -r USER-CCA@cca.in2p3.fr:/sps/qubic/Data/Calib-TD/2019-04-06 .`

in a terminal and getting a directory called `2019-04-06` (`2019-04-06`) in your current working directory. It tooks more than 20  minutes (hours) (depens on your network speed and others probably) and size aprox 2GB (13GB). 

**Calibration source data** taken from CCA: `/sps/qubic/Data/Calib-TD/calsource`

* If you are working in your personal computer, you can copy the dataset by running: 

&nbsp;&nbsp;&nbsp;&nbsp;`scp -r USER-CCA@cca.in2p3.fr:/sps/qubic/Data/Calib-TD/calsource/*20190406* .`

    in a terminal.


## 2022-04-14


1) Team: S. Torchinsky, M. Piat, C. Rodriguez, F. Rodriguez, G. Ramos, M. Platino

2) Tasks: 

2.1) We generated a map in Salta @170 GHz. The scanned region was +/- 20 deg in both az and el. The velocity of the scan was V14. No modulation on the calibration source

Note that the signal generator does not accept offset=0, so it was set to offset=0.001.  In fact, we should just turn off the modulator when we don't want modulation.

FLL PID=0,300,0

script:  ScanMapFastNoModulation.dscript

2.2) Calsource configuration:

Calsource: frequency=170.00GHz
modulator: frequency=N/A shape=dc amplitude=N/A offset=0.001Vdc duty_cycle=N/A
amplifier: filter_mode=12db_low_pass dynamic_range=low_noise gain=2000 filter_low_frequency=10.0 filter_high_frequency=0.3 coupling=dc invert=ON

3) Materials used: QUBIC Studio, QUBIC Central, QUBIC

4) Providers for the materials: APC.

5) Problems found: We had to use a value of 0.001 V for the offset setting of the CalS because it did not respond to seting the offset value to 0V.

6) Reference to a Procedure followed: N/A

In [ ]:
%matplotlib inline

# Importation modules
import os
import sys
import glob
import inspect
from importlib import reload

import scipy.ndimage.filters as f
import healpy as hp
import pickle
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
from matplotlib import rc
from pysimulators import FitsArray as fa

rc('figure', figsize=(15,8))
rc('font', size=12)
rc('text', usetex=False)
rc('image', cmap='viridis')

import qubic
from qubicpack.qubicfp import qubicfp
import qubic.io
from pysimulators import FitsArray
import qubic.fibtools as ft
import qubic.demodulation_lib as dl
import qubic.sb_fitting as sbfit
import qubic.SpectroImLib as si
import qubic.selfcal_lib as sc
import qubicpack as qp

# Super local module (tools)
import mapmaking_datafiles as md
# Super local module (plots)
import mapmaking_plots as mmp
import psutil
initram = psutil.virtual_memory()[0]/1e9 # in Gb
print("RAM available in GB %5.2f" %(initram))
print("RAM used in GB %5.2f" %(psutil.virtual_memory()[3]/1e9))

**0th)** Set local paths --> `Paths` dictionary containing all the directories needed to run the notebook

In [ ]:
def check_dir(dirfile):
    if os.path.isdir( dirfile): 
        print('TOD directory {} exists'.format(dirfile)) 
    else: 
        raise ValueError('dirfile {} is not an existing directory'.format(dirfile) )
        
if 'QUBIC_TODDIR' in os.environ.keys():
    qubic_toddir = os.environ['QUBIC_TODDIR']
    print("TOD directory: ",qubic_toddir)
    check_dir(qubic_toddir)
else:
    qubic_toddir = input("You have to specify a directory where the Time Ordered Data (TOD) are. \
    The usual format of TOD root directories is yyyy-mm-dd, e.g. /sps/qubic/Data/Calib-TD at CCA")
    print("Enter the directory where the TOD's are: ",qubic_toddir)
    check_dir(qubic_toddir)

In [ ]:
# Setup
verbose = True

#Directories
# Read data
reload(md)
confg = "new"
if confg == "new":
    day = '2020-07-30'
    keyword = '*2020*'
    #day = '2022-04-14'
    #keyword = '*ScanMap*'
    dirfile = None
    savedirtest = '/home/mgamboa/pruebas/qubicpipenew/'
elif confg == "old":
    #day = '2019-04-06'
    day = '2019-03-31'
    keyword = '*ScanMap*'
    dirfile = qubic_toddir + "calsource/"
    savedirtest = '/home/mgamboa/pruebas/qubicpipeold/'

# Directory where the raw TOD are save
data_dir = qubic_toddir + day + '/'

if not os.path.isdir(savedirtest):
    print("Creating {} directory to save demodulated TODs".format(savedirtest))
    os.mkdir(savedirtest)
    print("Done? {}".format(os.path.isdir(savedirtest)))
# Where to save the demodulated data 
savetoddemod = savedirtest + "Demod_TOD/"
saveflatmaps = savedirtest + "Flat/"
#create directory if None
if not os.path.isdir(savetoddemod):
    print("Creating {} directory to save demodulated TODs".format(savetoddemod))
    os.mkdir(savetoddemod)
    print("Done? {}".format(os.path.isdir(savetoddemod)))
else: 
    print("Demodulated TODs in")
    check_dir(savetoddemod)
if not os.path.isdir(saveflatmaps):
    print("Creating {} directory to save maps".format(saveflatmaps))
    os.mkdir(saveflatmaps)
    print("Done? {}".format(os.path.isdir(saveflatmaps)))
else:
    print("Flat maps in .fits format in")
    check_dir(saveflatmaps)

Paths = {"RawTodDir": data_dir,
         "parent_dir_work": savedirtest,
         "demod_tod_work": savetoddemod,
         "flat_maps_work": saveflatmaps,
         "calsource_data": dirfile}
#take files from raw data (TOD) diretory
dirs = np.sort(glob.glob(Paths["RawTodDir"] + keyword))

Paths.update({"sweeps_data": dirs})

**1st)** Use `fit_sb` from `demodulation_lib` module from `qubicsoft` package. 

[Local]Output files in: `/home/mgamboa/qubic/QUBIC/qubiccalibration/Data/150GHz-2019-04-06/[Flat, Healpix]`

script used: `Analyse-Scan-2020-01-15-MemorySave.Rmd` and `Sample_Demodulation`

In [ ]:
# Create focal plane object 
ram0_loc = psutil.virtual_memory()[3]/1e9 # in Gb

QubicFP = qubicfp()
sweep = 0
QubicFP.read_qubicstudio_dataset(Paths["sweeps_data"][sweep])

print("RAM used %5.2f Gb" %(psutil.virtual_memory()[3]/1e9 - ram0_loc))

### Read calibration data

In [ ]:
reload(md)
ram0_loc = psutil.virtual_memory()[3]/1e9 # in Gb

t_src, data_src = md.read_calsource_data(QubicFP, date = day,
                      keyword = keyword, datadir = Paths["RawTodDir"],
                      datacal = Paths["calsource_data"], verbose = True)

print("RAM used %5.2f Gb" %(psutil.virtual_memory()[3]/1e9 - ram0_loc))

In [ ]:
#Read azimuth and elevation from raw data
az = QubicFP.azimuth()
el = QubicFP.elevation()
#if verbose: QubicFP.read_qubicstudio_dataset(Paths["sweeps_data"][sweep])

In [ ]:
print(QubicFP.hk["CALSOURCE-CONF"]["Amp_lfreq"], 
      QubicFP.hk["CALSOURCE-CONF"]["Amp_hfreq"], 
      QubicFP.hk["CALSOURCE-CONF"])

In [ ]:
time_azel = QubicFP.timeaxis(datatype = 'hk', axistype = 'pps')
t0 = time_azel[0]

#### check plot ... ok

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(121)
plt.plot(time_azel, az, 'b-')
plt.plot(time_azel, el-np.mean(el), 'r-')
plt.yticks([])
plt.xlabel('tiempo [s]')
plt.subplot(122)
plt.plot(time_azel[400:16000], az[400:16000], 'b-')
plt.plot(time_azel[400:16000], el[400:16000]-np.mean(el), 'r-')
plt.yticks([])
plt.xlabel('tiempo [s]')


In [ ]:
reload(mmp)
mmp.plot_scan(time_azel, t_src, data_src, az, el)

In [ ]:
# Read the timeline to test the way to do it
TESNum = 94
asic = 1
tod_data = QubicFP.timeline(TES = TESNum, asic = asic)
tod_time = QubicFP.timeaxis(axistype = 'pps', asic = asic)

`check plot` TOD(tes) data and calsource ... ok

In [ ]:
reload(mmp)
mmp.plot_raw_data(tod_time, tod_data, t_src, data_src, TESNum = TESNum, asic = asic)

## Filtering data

In [ ]:
QubicFP.hk["CALSOURCE-CONF"]["Amp_lfreq"], QubicFP.hk["CALSOURCE-CONF"]["Amp_lfreq"][0]

In [ ]:
# Set parameters for harmonics of PT
#load the modulation frequency from the HK data
thefreqmod = QubicFP.hk['CALSOURCE-CONF']['Mod_freq']

period = 1./ thefreqmod
#Set parameters for filtering data
lowcut = QubicFP.hk["CALSOURCE-CONF"]["Amp_hfreq"][0]
highcut = QubicFP.hk["CALSOURCE-CONF"]["Amp_lfreq"][0]
# New spectra
# Using 10 harmonics
nharm = 10
#notch filter at each harmonic of the PTs
notch = np.array([[1.724, 0.005, nharm]])
#fourier_cuts = [lowcut, highcut, notch]

In [ ]:
plt.subplot(2,1,1)
plt.plot((tod_time - t0)/3600, dl.renorm(tod_data), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plt.plot((t_src - t0)/3600, dl.renorm(data_src)-5, label = 'source')
plt.xlabel('Hours')
plt.legend(loc = 'upper left')
plt.title("bla")
plt.ylim(-10,10)

plt.subplot(2,1,2)
tzoom = 1.2940
dtzoom = 0.005
oktime_d = np.abs((tod_time - t0)/3600-tzoom) < dtzoom
oktime_s = np.abs((t_src - t0)/3600-tzoom) < dtzoom
plt.plot((tod_time - t0)[oktime_d]/3600, dl.renorm(tod_data[oktime_d]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plt.plot((t_src[oktime_s]-t0)/3600, dl.renorm(data_src[oktime_s]), label='source')
plt.xlabel('Hours')
plt.legend(loc='upper left')
#plt.title("bla")

In [ ]:
# Power spectra without filtering (raw power spectra of TOD)
# spectrum, freq = old "spectrum_f", "freq_f"
ram0_loc = psutil.virtual_memory()[3]/1e9

spectrum_raw, freq_raw = ft.power_spectrum(tod_time, tod_data, 
                                       rebin = True)
#tod_data_filtered = old "newdata"
tod_data_filtered = ft.filter_data(tod_time, tod_data, lowcut, highcut, 
                         notch = notch, rebin = True, 
                         verbose = True, order = 5)
# spectrum_filtered, freq_filtered ex "spectrum_f2", "freq_f2"
spectrum_filtered, freq_filtered = ft.power_spectrum(tod_time, tod_data_filtered, rebin=True)

print("RAM used %5.2f Gb" %(psutil.virtual_memory()[3]/1e9 - ram0_loc))

tod_data_filtered2 = ft.filter_data(tod_time, tod_data, 0.03, 10, 
                         notch = notch, rebin = True, 
                         verbose = True, order = 5)
# spectrum_filtered, freq_filtered ex "spectrum_f2", "freq_f2"
spectrum_filtered2, freq_filtered2 = ft.power_spectrum(tod_time, tod_data_filtered2, rebin=True)

`check plot` spectras for RAW data and filtered data

In [ ]:
reload(mmp)

mmp.plot_spectra_comparisson(freq_raw, spectrum_raw, freq_filtered, 
                             spectrum_filtered, period, lowcut, highcut, notch)

In [ ]:
plt.subplot(211)
plt.xscale("log")
plt.yscale("log")
plt.plot(freq_raw, spectrum_raw)
plt.plot(freq_filtered, spectrum_filtered, label = "freq from hk")
plt.plot(freq_filtered2, spectrum_filtered2, label = "manual values")
plt.xlim(1e-2,9e1)
plt.ylim(1e1, 1e17)
plt.legend()
plt.subplot(212)
plt.xscale("log")
plt.yscale("log")
plt.plot(freq_filtered2, abs(spectrum_filtered - spectrum_filtered2), 'r-')
plt.xlim(1e-2,9e1)
plt.ylim(1e1, 1e17)


`Check plot` comparing clasoruce data, raw and filtered tod... okk

In [ ]:
reload(mmp)
#kwargs_plot = {"xmin": 0, "xmax": max(tod_time),
#               "ymin": -5, "ymax": 5}
mmp.plot_data_and_src(tod_time, tod_data, 
                      tod_data_filtered,
                      t_src, data_src,
                     )#**kwargs_plot)
mmp.plot_data_and_src(tod_time, tod_data, 
                      tod_data_filtered2,
                      t_src, data_src)


In [ ]:
reload(md)

##### Filtering to be used
demod_kwvars = {"method": "demod_quad",
                "remove_noise": False}

demodulate = False

ram0_loc = psutil.virtual_memory()[3]/1e9
newt_demod, amps_demod = md.pipe_demodulation(QubicFP, t_src, data_src,
                                             savetoddemod,
                                             demodulate = demodulate,
                                             verbose = True,
                                             **demod_kwvars)
print("RAM used %5.2f Gb" %(psutil.virtual_memory()[3]/1e9 - ram0_loc))


In [ ]:
QubicFP.azimuth(), QubicFP.elevation(), QubicFP.hk["INTERN_HK"]["Platform-Azimut"]

In [ ]:
reload(mmp)
mmp.plot_synchronizated_data(tod_time, t_src, tod_data_filtered, data_src)

In [ ]:
print("RAM available in GB %5.2f" %(psutil.virtual_memory()[0]/1e9))
print("RAM used in GB %5.2f" %(psutil.virtual_memory()[3]/1e9))

In [ ]:
azindex = (naz * (newaz - azmin) / (azmax - azmin)).astype(int)
elindex = (nel * (newel - elmin) / (elmax - elmin)).astype(int)

# ## Keeping only the inner part
inside = (azindex >= 0) & (azindex < naz) & (elindex >= 0) & (elindex < nel)

In [ ]:
time_azel = QubicFP.timeaxis(datatype = 'hk', axistype = 'pps')

azmin = min(az)
azmax = max(az)
elmin = min(el)
elmax = max(el)

ComputeFlatMaps = True
if ComputeFlatMaps:
    newaz = np.interp(newt_demod, time_azel, az)
    newel = np.interp(newt_demod, time_azel, el)

    naz = 101
    nel = 101
    nmaps = 256
    mapdata = np.zeros((nmaps, nel, naz))

    mapdata, azmap, elmap = dl.coadd_flatmap(amps_demod, newaz, newel, 
                                            filtering = None,
                                            azmin = azmin, azmax = azmax, 
                                            elmin = elmin, elmax = elmax, 
                                            naz = naz, nel = nel)
    FitsArray(azmap).save(savetoddemod + 'azimuth.fits')    
    FitsArray(elmap).save(savetoddemod + 'elevation.fits')    
    for i in range(256):
        FitsArray(mapdata[i,:,:]).save(saveflatmaps + 'imgflat_TESNum_{}.fits'.format(i+1))  
    
elif not ComputeFlatMaps:
    azmap = np.array(FitsArray(savetoddemod + 'azimuth.fits'))
    elmap = np.array(FitsArray(savetoddemod + 'elevation.fits'))
    mapdata = np.zeros((256, len(azmap), len(elmap)))
    for i in range(256):
        mapdata[i,:,:] = np.array(FitsArray(saveflatmaps + 'imgflat_TESNum_{}.fits'.format(i+1)))

In [ ]:
#Look at one map
TESNum = 93
ites = TESNum - 1
mm, ss = ft.meancut(mapdata[ites,:,:],3)
plt.imshow(mapdata[ites,:,:], aspect = 'equal',
       #extent = [np.cos(np.radians(50)) * azmin, 
       #          np.cos(np.radians(50)) * azmax, 
       #          elmin, 
       #          elmax],
          )#vmin = 0, vmax = 2e6)
plt.title('TES {}'.format(ites+1))

In [ ]:
blil = sbfit.get_flatmap(93, "/home/mgamboa/qubic/QUBIC/qubiccalibration/Data/150GHz-2019-04-06/", 
                        azmin = azmin, azmax = azmax, 
                        remove = None)[0]
blil2 = sbfit.get_flatmap(93, "/home/mgamboa/pruebas/qubicpipe/", 
                        azmin = azmin, azmax = azmax, 
                        remove = None)[0]

In [ ]:
blil.shape

In [ ]:
plt.subplot(121)
plt.imshow(blil, vmin = 0, 
           vmax = 1e5, cmap = "bwr")
plt.subplot(122)
plt.imshow(blil2, vmin = 0, 
           vmax = 1e5, cmap = "bwr")
plt.tight_layout()

All TES. Save flat maps in files. 

from James's notebook `Examples_notebooks/Sample_Demodulation.Rmd`

carefull with the demodulation, sometimes the the asics will have different demodulate lengths which may cause an issue here. the simple solution is to demodulate each asic separately and combine later

doing this part can probably be improved. I think with jupyter only one core is used. Maybe somebody already knows how to multithread with jupyter, please tell!

In [ ]:
"""save the maps
you can see the numbering here doesn't really reflect the real naming of TES we use"""

for i in range(256):
    print("saving flat map for tes {}".format(i+1))
    FitsArray(mapdata[i,:,:]).save(savedirtest+'Flat/imgflat_TESNum_{}.fits'.format(i+1))    
    FitsArray(mapdata[i,:,:]).save(savedirtest+'Flat/imgflat_TESNum_{}.fits'.format(i+1))

Test load the maps...ok!

flattest = np.zeros([nmaps, nel, naz])
flattest[0,:,:], az, el = sbfit.get_flatmap(256, savedirtest, 
                                     azmin = azmin, azmax = azmax, 
                                     remove = None)
imshow(flattest[0,:,:], aspect = 'equal',
       extent = [np.cos(np.radians(50)) * azmin, 
                 np.cos(np.radians(50)) * azmax, 
                 elmin, 
                 elmax],
      vmin=0, vmax=2.e6)

**2nd)** Once we have the flat/*helapy* maps we fit the position of the peaks for each TES. To do that we can select between fitting symetric gaussian or asymetric (`sym` or `asym`) for each peak.

In [ ]:
keys_sym = ['xc', 'yc', 'dist', 'angle', 'distx', 'disty', 'ampgauss', 
            'xcgauss', 'ycgauss', 'fwhmgauss', 'fwhmpeaks']

keys_asym = ['xc', 'yc', 'dist', 'angle', 'distx', 'disty', 'ampgauss', 
            'xcgauss', 'ycgauss', 'fwhmgauss']
for i in range(9):
    keys_asym.append('fwhmxpeak_{}'.format(i))
for i in range(9):
    keys_asym.append('fwhmypeak_{}'.format(i))
for i in range(9):
    keys_asym.append('rhopeak_{}'.format(i))

In [ ]:
#import debugging_fit as df
#reload(df)
#flat_map, azinit, elinit, fitout, xxyynew = df.fit_sb(93, savedirtest)

### I couldn't make this work but it works for JCh so I read the fit of the synthesized beam from files

In [ ]:
#import debugging_fit as df

sbfitmodel = sbfit.SbModelIndepPeaks(nrings=2, common_fwhm=True, 
                                 no_xy_shift=False, distortion=False)
detnums = [1,2,3,4]
if verbose: 
    print('Using Fit Model {} for TES #{}'.format(sbfitmodel.name,detnums[i]))

flattest, az, el = sbfit.get_flatmap(detnums[1], savedirtest, azmin = azmin, azmax = azmax, remove = None)
figure()
fit, thexypeaks = sbfit.fit_sb(flattest, az, el, sbfitmodel, resample=False, newsize=70,
                               verbose=True, doplot=True, 
                               extra_title='TES #{}'.format(detnums[1]))

In [ ]:
# Read the data
for TESNum in range(1, 257):
    print(TESNum)
    flatmap, az, el = sbfit.get_flatmap(TESNum, savedirtest, remove=None, azmin=azmin, azmax=azmax)

    #### Instantiate a model
    sbfitmodel3 = sbfit.SbModelIndepPeaks(nrings=2, common_fwhm=True, no_xy_shift=False, distortion=False)
    
    fit, xypeaks = sbfit.fit_sb(flatmap, az, el, sbfitmodel3, verbose=False, resample=False, 
                            doplot=False, extra_title='TES #{}'.format(TESNum))

In [ ]:
GaussianShape = 'sym'
if GaussianShape == 'sym':
    funcfit = dl.fit_sb
elif GaussianShape == 'asym':
    funcfit = dl.fit_sb_asym

In [ ]:
flatmap_init, az_init, el_init = sbfit.get_flatmap(2, dirfiles)
flatmap = flatmap_init
az = az_init
el = el_init
az2d, el2d = np.meshgrid(az * np.cos(np.radians(50)), np.flip(el))
distance_max = 5.
az_center=0.
el_center=50.
scaling=140e3
mask = (np.sqrt((az2d - az_center) ** 2 + (el2d - el_center) ** 2) < distance_max).astype(int)
wmax = np.where((flatmap * mask) == np.max(flatmap * mask))
maxval = flatmap[wmax][0]
x = [az2d, el2d]
parsinit = np.array([az2d[wmax][0], el2d[wmax][0], 8.3, 44., 0., 0.009, maxval / scaling, 0., 50., 13., 1.])
rng = [[az2d[wmax][0] - 1., az2d[wmax][0] + 1.],
           [el2d[wmax][0] - 1., el2d[wmax][0] + 1.],
           [8., 8.75],
           [43., 47.],
           [-0.02, 0.02],
           [-0.02, 0.02],
           [0, 1000],
           [-3, 3],
           [47., 53],
           [10., 16.],
           [0.5, 1.5]]


In [ ]:
fit = ft.do_minuit(x, np.ravel(flatmap / scaling), np.ones_like(np.ravel(flatmap)), parsinit,
                       functname=dl.flattened_qubic_sb_model, chi2=ft.MyChi2_nocov, rangepars=rng,
                       force_chi2_ndf=True)


In [ ]:
machine = os.uname()[1]
dirfreq = "150GHz-2019-04-06/"
if machine == 'shaka':
	dirallfreqs = "/home/mgamboa/qubic/QUBIC/qubiccalibration/Data/"
elif machine == 'hain':
	dirallfreqs = "/home/martin/QUBIC/qubiccalibration/Data/Calib-TD/"

dirfiles = dirallfreqs+dirfreq
dirsave = dirallfreqs+dirfreq+"FitSB/"

ft.printnow("====================================")
ft.printnow("====================================")
ft.printnow("Starting \n")
ft.printnow("====================================")
ft.printnow("====================================")

t0 = time.time()

for TESIndex in range(2):
    TESNum = TESIndex+1
    if GaussianShape == 'sym':
        flatmap, az, el, fit, xypeaks = dl.fit_sb(TESNum, dirfiles)
    elif GaussianShape == 'asym':
        flatmap, az, el, fit, xypeaks = dl.fit_sb_asym(TESNum, dirfiles)
    
    FitsArray(fit[1]).save(dirsave+'/fit_sb_{}_TESNum_{}.fits'.format(
        GaussianShape, TESNum))
    FitsArray(fit[2]).save(dirsave+'/errfit_sb_{}_TESNum_{}.fits'.format(
        GaussianShape, TESNum))
    ft.printnow("Done (sym) TES = {}".format(TESIndex+1))
    
t1 = time.time()
ft.printnow("====================================")
ft.printnow("====================================")
ft.printnow("Sym done in {:.2f} \n".format((t1 - t0)/60  ))
ft.printnow("====================================")
ft.printnow("====================================")

t2 = time.time()


**3rd)** Once we had create the flat and healpix maps, we have our demodulated data. We now read it doing

In [ ]:
# INSTRUMENT
ns = 256
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_oneDet_multiband.dict')

print(d['kind'])
d['kind']='I'
print(d['kind'])
d['nside']=ns

fittedpeakfile = os.environ['QUBIC_TODDIR'] + '150GHz-2019-04-06/FitSB/fitted_peaks.fits'
directory = os.environ['QUBIC_TODDIR'] + '150GHz-2019-04-06/'

In [ ]:
c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50
flatmap, az, el = sbfit.get_flatmap(1, directory, azmin=azmin, azmax=azmax)
npix = len(np.ravel(flatmap))
alldata_renorm = np.zeros((256, npix))
for i in range(256):
    flatmap, az, el = sbfit.get_flatmap(i+1, directory, azmin=azmin, azmax=azmax)
    mm,ss = ft.meancut(flatmap,3)
    alldata_renorm[i,:] = np.ravel((flatmap-mm)/ss)  

med = np.reshape(np.median(alldata_renorm,axis=0),np.shape(flatmap))
mm = np.mean(med)
ss = np.std(med)

plt.rc('figure', figsize=(15,8))
plt.imshow(np.reshape(med, np.shape(flatmap)),
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], aspect='equal')
plt.colorbar()
plt.title('Median over all S.B. images (renormalized)')